In [47]:
import openai

from configs.config import arg_config_dev as config
import pandas as pd
import time
import numpy as np
import os
openai.api_key = config['openai_api_key']
openai.organization = config['openai_organization']

In [48]:
# data_path = os.path.join(params['data_path'], 'xzp_la_album_recable_{dt}.csv'.format(dt=dt))
data_path = 'xiaoshuo.csv'
album_df = pd.read_csv(data_path, ',', error_bad_lines=False)

/var/folders/7y/hs0h3vks5wsc81xrqpc8m8q40000gp/T/ipykernel_31776/2565044820.py:3: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  album_df = pd.read_csv(data_path, ',', error_bad_lines=False)
/var/folders/7y/hs0h3vks5wsc81xrqpc8m8q40000gp/T/ipykernel_31776/2565044820.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  album_df = pd.read_csv(data_path, ',', error_bad_lines=False)


In [49]:
album_df

,albumid,albumname,tagname2,tagname3,play_dur,uv,rn
0,42540805,校花的贴身高手,有声小说1,都市纵横,467687.05,33986,1
1,58273021,青岛江湖往事-聂磊传奇,有声小说1,都市纵横,221704.93,29790,2
2,37191013,盗墓笔记,有声小说1,悬疑推理,119166.38,29688,3
3,39843428,斗罗大陆,有声小说1,玄幻异界,129491.52,29509,4
4,65637931,修炼9999级了，老祖才100级,有声小说1,玄幻异界,22354.11,25225,5
...,...,...,...,...,...,...,...
3584,57384454,修仙女配只想独自美丽,有声小说1,玄幻异界,483.43,101,3570
3585,38338851,贞观贤王,有声小说1,历史铁血,621.39,101,3570
3586,40462658,我真的不是气运之子,有声小说1,玄幻异界,830.04,101,3570
3587,56059393,坏蛋是怎么炼成的,有声小说1,都市纵横,263.29,101,3570


In [50]:
album_df['albumname'].fillna('', inplace=True)
album_df['albumname_clean'] = album_df['albumname'].apply(
        lambda x: x.replace('@BI_COLUMN_SPLIT@', ' ').replace('@BI_FIELD_R@', ' ').replace('@BI_ROW_SPLIT@',
                                                                                           ' ').replace('\u3000',
                                                                                                        ' '))  # 去除制表符、空白符

In [51]:
album_df

,albumid,albumname,tagname2,tagname3,play_dur,uv,rn,albumname_clean
0,42540805,校花的贴身高手,有声小说1,都市纵横,467687.05,33986,1,校花的贴身高手
1,58273021,青岛江湖往事-聂磊传奇,有声小说1,都市纵横,221704.93,29790,2,青岛江湖往事-聂磊传奇
2,37191013,盗墓笔记,有声小说1,悬疑推理,119166.38,29688,3,盗墓笔记
3,39843428,斗罗大陆,有声小说1,玄幻异界,129491.52,29509,4,斗罗大陆
4,65637931,修炼9999级了，老祖才100级,有声小说1,玄幻异界,22354.11,25225,5,修炼9999级了，老祖才100级
...,...,...,...,...,...,...,...,...
3584,57384454,修仙女配只想独自美丽,有声小说1,玄幻异界,483.43,101,3570,修仙女配只想独自美丽
3585,38338851,贞观贤王,有声小说1,历史铁血,621.39,101,3570,贞观贤王
3586,40462658,我真的不是气运之子,有声小说1,玄幻异界,830.04,101,3570,我真的不是气运之子
3587,56059393,坏蛋是怎么炼成的,有声小说1,都市纵横,263.29,101,3570,坏蛋是怎么炼成的


In [52]:
index2albumid = album_df['albumid'].tolist()
index2album = album_df['albumname_clean'].tolist()

gpt_model = {
    'curie': 'text-curie-001',
    'davinci': 'text-davinci-003',
    'gpt': 'gpt-3.5-turbo'
}

In [53]:
len(index2album)

3589

['贞观贤王', '修仙女配只想独自美丽', '都市之修真归来', '大染坊丨民国商战', '央视87版《红楼梦》', '百妖谱']

In [82]:
a=index2album[1700:1710]
print(a)
b=index2albumid[1700:1710]

['史上第一妖后', '乡村小神医', '民间故事：悬疑民俗故事', '碧血剑', '加代传奇', '我当阴阳先生的那几年 全集', '经典传奇2013', '超级战兵', '末日丧尸进化丨免费有声小说', '剑来精品多人有声剧']


In [83]:
with open('./openai_answer.txt', "a+", encoding='utf-8') as f:
    for index,albumname in enumerate(a):
        # if index <10:
        #     continue
        prompt = "'《{}》'这本小说的标签有哪些？至少返回6个，不需要换行，用逗号分开".format(albumname)
        completion = openai.ChatCompletion.create(
            model=gpt_model['gpt'],
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        text_raw = completion.choices[0].message['content']
        text = text_raw.strip()
        f.writelines("{}:{}:{}\n".format(b[index], albumname, text))
        print("Human: {}".format(prompt))
        print("AI: {}".format(text))
        time.sleep(2)

Human: '《史上第一妖后》'这本小说的标签有哪些？至少返回6个，不需要换行，用逗号分开
AI: 古代言情, 重生, 宫斗, 女强, 美食, 心机
Human: '《乡村小神医》'这本小说的标签有哪些？至少返回6个，不需要换行，用逗号分开
AI: 乡村, 小说, 小神医, 医学, 穿越, 古代
Human: '《民间故事：悬疑民俗故事》'这本小说的标签有哪些？至少返回6个，不需要换行，用逗号分开
AI: 悬疑,民俗,故事,民间传说,神秘,传统文化
Human: '《碧血剑》'这本小说的标签有哪些？至少返回6个，不需要换行，用逗号分开
AI: 武侠、江湖、架空历史、爱情、恩怨、儒家思想。
Human: '《加代传奇》'这本小说的标签有哪些？至少返回6个，不需要换行，用逗号分开
AI: 历史,文学,日本,宫廷,争斗,传奇
Human: '《我当阴阳先生的那几年 全集》'这本小说的标签有哪些？至少返回6个，不需要换行，用逗号分开
AI: 玄幻,仙侠,都市,鬼怪,异能,修真
Human: '《经典传奇2013》'这本小说的标签有哪些？至少返回6个，不需要换行，用逗号分开
AI: 武侠,历史,传奇,冒险,魔幻,修真
Human: '《超级战兵》'这本小说的标签有哪些？至少返回6个，不需要换行，用逗号分开
AI: 未来,战争,科幻,奇幻,机甲,人类与机器的对抗
Human: '《末日丧尸进化丨免费有声小说》'这本小说的标签有哪些？至少返回6个，不需要换行，用逗号分开
AI: 灾难, 生存, 前线, 进化, 末日, 恐怖
Human: '《剑来精品多人有声剧》'这本小说的标签有哪些？至少返回6个，不需要换行，用逗号分开
AI: 玄幻,修仙,异界,战斗,情感,多人听书


In [ ]:
# 1. 必须直接使用作品名，不能加xxx精品播放之类的（可以通过分隔符处理）
# 2. 网速问题
# 3. 账号收费问题
# 4. 越经典作品的越准确